In [ ]:
import mysql.connector
import pandas as pd
import requests
import json
from datetime import date
import datetime
import time

### Download Data from the Polygon

- Meta (META)
- Apple (APPL)
- Amazon (AMZN)
- Netflix (NFLX)
- Alphabet (GOOG)
- Microsoft (MSFT)

In [ ]:
companies = ['META','AAPL','AMZN','NFLX','GOOG','MSFT']  #company symbols to retrieve.
stock_date = '2023-02-07'

In [ ]:
test_date = datetime.datetime.strptime("2022-12-01", "%Y-%m-%d")
date_range=pd.date_range(test_date, periods = 31)

date_range=list(date_range.astype('str'))

In [ ]:
#You need to set up your connection with your credentials
connection = mysql.connector.connect(user='stock_user', password='letmein',
                              host='127.0.0.1',
                              database='stock_data')
global_cursor = connection.cursor()

In [ ]:
def get_company_data(ticker, timestamp, cnx, local_cursor):
    request_string = 'https://api.polygon.io/v1/open-close/'+ticker+'/'+timestamp+'?adjusted=true&apiKey=46ipIrH5BrzkTqZj7OUvik7XAFOpu8GJ'
    api_response = requests.get(request_string)
    data = api_response.text
    parse_json = json.loads(data)
    db_row=[(v) for k, v in parse_json.items()]
    
    #for item in db_row:
    #    print(item)
    
    if(db_row[0] != 'NOT_FOUND'):
    
        symbol = db_row[2]
        ticker_date = db_row[1]
        open_price = db_row[3]
        high_price = db_row[4]
        low_price = db_row[5]
        close_price = db_row[6]
        volume = db_row[7]

        sql = "INSERT INTO stock (symbol, ticker_date, open_price, high_price, low_price, close_price, volume) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        val = (symbol, ticker_date, open_price, high_price, low_price, close_price, volume)
        local_cursor.execute(sql, val)

        cnx.commit()

        #print(local_cursor.rowcount, "record inserted.")
    
    time.sleep(15)

In [ ]:
#for company in companies:
#    print("Gettting Data for ", company)
#    get_company_data(company, stock_date,connection, global_cursor)

In [ ]:
for company in companies:
    print("Gettting Data for: ", company)
    for day in date_range:
        print("Date: ", day)
        get_company_data(company, day,connection, global_cursor)

In [ ]:
ticker='META'
timestamp='2022-12-03'
request_string = 'https://api.polygon.io/v1/open-close/'+ticker+'/'+timestamp+'?adjusted=true&apiKey=46ipIrH5BrzkTqZj7OUvik7XAFOpu8GJ'
api_response = requests.get(request_string)
data = api_response.text
parse_json = json.loads(data)

In [ ]:
parse_json

In [ ]:
db_row=[(v) for k, v in parse_json.items()]

In [ ]:
db_row[0]

In [ ]:
connection.close()